# Mision To Mars

In [46]:
%reload_ext lab_black

In [50]:
import requests
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

# Reflect Tables into SQLAlchemy ORM

In [64]:
# Define NASA News url
NASA_URL = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

driver = webdriver.Firefox()
driver.get(NASA_URL)
driver.implicitly_wait(10)
nasa_html = driver.page_source
driver.close()

# Loading page
nasa_soup = BeautifulSoup(nasa_html, "html.parser")

In [105]:
# Extract titles and text from divs
news_title = [
    tag.text
    for tag in [
        li for ul in nasa_soup for li in ul.findAll("div", class_="content_title")[0]
    ]
]

news_p = [
    tag.text
    for tag in [
        li for ul in nasa_soup for li in ul.findAll("div", class_="article_teaser_body")
    ]
]

image_url = [
    "https://mars.nasa.gov"
    + tag.img["src"].replace("list_view", "main").replace("320x240", "web")
    for tag in [li for ul in nasa_soup for li in ul.findAll("div", class_="list_image")]
]

nasa_result = list(zip(news_title, news_p, image_url))
nasa_result

[("A Year of Surprising Science From NASA's InSight Mars Mission",
  "A batch of new papers summarizes the lander's findings above and below the surface of the Red Planet.",
  'https://mars.nasa.gov/system/news_items/main_images/8613_InSight-Nature-papers-web.jpg')]